In [1]:
!nvidia-smi

Wed May 22 23:00:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:11:00.0  On |                  N/A |
| 37%   49C    P5              42W / 350W |    519MiB / 24576MiB |     38%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import psutil

# Get the total and available memory
memory_info = psutil.virtual_memory()

total_memory = memory_info.total / (1024 ** 3)  # Convert from bytes to GB
available_memory = memory_info.available / (1024 ** 3)  # Convert from bytes to GB

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")

Total Memory: 23.38 GB
Available Memory: 17.89 GB


In [9]:
from peft import PeftModel, PeftConfig
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch
from PIL import Image
import requests

# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model configurations
config = PeftConfig.from_pretrained("firqaaa/vsft-llava-1.5-7b-hf-liveness-trl")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
base_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf",
                                                               torch_dtype=torch.float16,
                                                               low_cpu_mem_usage=True,
                                                               device_map="auto",
                                                               load_in_4bit=True,
                                                               attn_implementation="flash_attention_2")
model = PeftModel.from_pretrained(base_model, "firqaaa/vsft-llava-1.5-7b-hf-liveness-trl")
model.to(device)

# url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
# image = Image.open(requests.get(url, stream=True).raw)

image_path = "/home/firqaaa/Python/Retired-Yann-LeCun/silicone_frames/5/frame7.jpg"
image = Image.open(image_path)

prompt = """USER: <image>\nI ask you to be an liveness image annotator expert to determine if an image "Real" or "Spoof". 
If an image is a "Spoof" define what kind of attack, is it spoofing attack that used Print(flat), Replay(monitor, laptop), or Mask(paper, crop-paper, silicone)?
If an image is a "Real" or "Normal" return "No Attack". 
Whether if an image is "Real" or "Spoof" give an explanation to this.
Return your response using following format :

Real/Spoof : 
Attack Type :
Explanation :\nASSISTANT:"""

# Prepare inputs and move to device
inputs = processor(prompt, images=image, return_tensors="pt").to(device)

# Generate output
output = model.generate(**inputs, max_new_tokens=300)

print("Response :")
# Decode and print the output
decoded_output = processor.decode(output[0], skip_special_tokens=True).split("ASSISTANT:")[-1].strip()
print(decoded_output)

Using device: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/firqaaa/anaconda3/envs/Research/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:103: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Response :
Real/Spoof : Spoof
Attack Type : Mask (silicone)
Explanation : The image shows signs of being a spoof attack. The face appears unnaturally smooth and lacks the natural texture and contours of a real human face. Additionally, the edges around the face and the overall appearance suggest that a mask made of silicone or a similar material has been used to spoof the image.
